In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer

In [2]:
keyword = "대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=1000)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 대통령
Scroll idx : 1 (100 docs)
Scroll idx : 2 (100 docs)
Scroll idx : 3 (100 docs)
Scroll idx : 4 (100 docs)
Scroll idx : 5 (100 docs)
Scroll idx : 6 (100 docs)
Scroll idx : 7 (100 docs)
Scroll idx : 8 (100 docs)
Scroll idx : 9 (100 docs)
Scroll idx : 10 (100 docs)
Scroll idx : 11 (100 docs)
Scroll idx : 12 (100 docs)
Scroll idx : 13 (100 docs)
Scroll idx : 14 (100 docs)
Scroll idx : 15 (100 docs)
Scroll idx : 16 (100 docs)
Scroll idx : 17 (100 docs)
Scroll idx : 18 (100 docs)
Scroll idx : 19 (100 docs)
Scroll idx : 20 (100 docs)
Scroll idx : 21 (100 docs)
Scroll idx : 22 (100 docs)
Scroll idx : 23 (100 docs)
Scroll idx : 24 (100 docs)
Scroll idx : 25 (100 docs)
Scroll idx : 26 (41 docs)
Trim lower 70% of docs (762 docs are cut)
Total retrieved Doc # :  1778



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

1763

In [10]:
doc_graph_anal = DocGraphAnalyzer(keyword)
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=5)

Doc #  : 1763
Node # : 987
Edge # : 166657

Community 0 :  267 Keyword, (검찰/조사/수사/관련/혐의)
Community 1 :  337 Keyword, (관계/북한/위원/청와대/대북)
Community 2 :  298 Keyword, (정부/지난/국민/정치/국회)
Community 3 :   85 Keyword, (대표/한국/이날/자유/바른미래당)



In [11]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]

In [12]:
top_doc_info_list

[[{'newsTitle': 'MB 측근 친인척도 줄줄이 이시형 등 무더기 사법처리할 듯',
   'extContent': '7일 법조계에 따르면 검찰이 이 전 대통령에게 소환을 통보하기까지 범죄 연루 정황이나 혐의점을 두고 수사를 벌인 측근과 친인척만 해도 20명을 훌쩍 넘어선다. 이들 중 상당수는 불법 자금수수 등과 관련해 이 전 대통령을 돕거나 지시를 받아 범죄에 가담한  공범  내지  종범 으로 재판에 넘겨질 가능성이 커 보인다. 친인척 중에서는 부인인 김윤옥 여사를 시작으로 아들 이시형씨, 큰형 이상은 다스 회장과 작은형 이상득 전 의원, 조카 이동형 김동혁씨, 사위 이상주 삼성전자 전무 등이 그간 검찰의 수사 선상에 올랐다. 각종 의혹 사건에 연루된 측근들의 면면을 훑어봐도 중요 인물들을 망라한 듯한 인상을 준다. 이 전 대통령의  멘토 로 불린 최시중 전 방송통신위원장부터 재산관리 등 궂은일을 맡은  가신  김백준 전 청와대 총무기획관 이병모 청계재단 사무국장까지 범위가 넓다. 이 전 대통령의 친인척과 측근들이 검찰의 수사망에 포착된 첫 사건은 국가정보원 특수활동비 상납 의혹이다. 이 전 대통령의 작은형 이상득 전 의원이 이미 특활비 1억원을 수수한 혐의로 검찰 조사를 받았다. 김백준 전 기획관과 김진모 전 비서관 등 이명박 정부 시절 청와대에 몸담았던 측근들이 특활비 사건 때문에 구속기소 됐다. 장다사로 전 총무기획관도 구속은 면했으나 특활비 상납 사건으로 재판에 넘겨지는 상황을 피하기 어려워 보인다. 이 밖에도 특활비를 받은 박재완 전 정무수석, 돈을 건넨 김성호 원세훈 전 국정원장 등이 검찰 조사를 받았다. 한편 특활비 의혹으로 조사를 받은 김희중 전 제1부속실장이 김윤옥 여사 측에 1억원 가량을 건넨 사실을 진술함에 따라 김 여사에게도 파장이 미쳤다. 다만 검찰은 아직 김 여사에 대한 수사 가능성에는 신중한 태도를 견지하고 있다. 이상득 전 의원은 2007년 대선을 전후로 이 전 대통령 측이 민간 부문에서 불법 자금을 받은 의혹과 관련해서도